In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plot
import matplotlib.pyplot as plt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Goal
 Predict total sales for every product and store in the next month

# Load dataset

In [ ]:
#sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
#test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
#sample_submission.csv - a sample submission file in the correct format.
#items.csv - supplemental information about the items/products.
#item_categories.csv  - supplemental information about the items categories.
#shops.csv- supplemental information about the shops.

#Converted dataset (English)
df_item_en=pd.read_csv('/kaggle/input/english-converted-datasets/items.csv')
df_submission_en=pd.read_csv('/kaggle/input/english-converted-datasets/sample_submission.csv')
df_item_cat_en=pd.read_csv('/kaggle/input/english-converted-datasets/item_categories.csv')
df_sale_train_en=pd.read_csv('/kaggle/input/english-converted-datasets/sales_train.csv')
df_shop_en=pd.read_csv('/kaggle/input/english-converted-datasets/shops.csv')
df_test_en=pd.read_csv('/kaggle/input/english-converted-datasets/test.csv')
#Competition dataset (Russian)
df_item=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
df_item_cat=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_sale_train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
df_shop=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
df_test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

# Datasets size

In [ ]:
print('items En ' +str(len(df_item_en)))
print('submission En ' +str(len(df_submission_en)))
print('items category En ' +str(len(df_item_cat_en)))
print('sales train En ' +str(len(df_sale_train_en)))
print('shops En ' +str(len(df_shop_en)))
print('tests En ' +str(len(df_test_en)))
print ('----------------------------------------')
print('items ' +str(len(df_item)))
print('submission ' +str(len(df_submission)))
print('items category ' +str(len(df_item_cat)))
print('sales train ' +str(len(df_sale_train)))
print('shops ' +str(len(df_shop)))
print('tests ' +str(len(df_test)))

## Data Wrangling

In [ ]:
df_item.head(1)
print(df_test.columns)
print(df_sale_train.columns)
print(df_submission.columns)
#extra information datasets
print(df_item.columns)
print(df_item_cat.columns)
print(df_shop.columns)

In [ ]:
print(df_test.head(1))
print(df_sale_train.head(1))
print(df_submission.head(1))

In [ ]:
#df_item_cat_en.head(1)

In [ ]:
#investigate the sales train dataset
#convert the date fiedl from string to datetime
df_sale_train['date'] = pd.to_datetime(df_sale_train['date'])
df_sale_train.head(10)

In [ ]:
#get the sales per items
df_sale_train["sales"] = df_sale_train["item_cnt_day"] * df_sale_train["item_price"]

In [ ]:
#by month - aggregate data in month level
df_sale_train['date'] = df_sale_train['date'].dt.year.astype('str') +'-' + df_sale_train['date'].dt.month.astype('str') + '-01' 
df_sale_train['date'] = pd.to_datetime(df_sale_train['date'])
#group by at month level - should we aggregate by shop??? as well
df_sale_train = df_sale_train.groupby('date').sales.sum().reset_index()

In [ ]:
df_total_price = df_sale_train.copy()
df_total_price["total_price"] = df_sale_train["item_cnt_day"] * df_sale_train["item_price"]
#df_total_price.groupby("item_price").sum().sort_values("total_price", ascending=False).plot(kind='bar')
df_total_price.groupby("item_price").sum()

In [ ]:
df_sale_train.head(10)

In [ ]:
#data merge
df_item_cat_all = df_item.merge(df_item_cat_en, left_on='item_category_id', right_on='item_category_id')
df_test_item = df_test.merge(df_item_cat_all, left_on='item_id', right_on='item_id')
df_test_item_shop = df_test_item.merge(df_shop_en, left_on='shop_id', right_on='shop_id')

# Data Transformation

In [ ]:
#plot sales monhtly

import matplotlib.pyplot as plt

# gca stands for 'get current axis'
ax = plt.gca()

df_sale_train.plot(kind='line',x='date',y='sales',ax=ax)

plt.show()


In [ ]:
#get the difference of sales compare to the previous month
df_sales_previous = df_sale_train.copy()
df_sales_previous['diff'] = df_sales_previous['sales'].shift(1)
#drop nulle values
df_sales_previous = df_sales_previous.dropna()
df_sales_previous['diff_previous'] = (df_sales_previous['sales'] - df_sales_previous['diff'])
df_sales_previous.head(10)

In [ ]:
#plot the diff with the previous month
import matplotlib.pyplot as plt

# gca stands for 'get current axis'
ax = plt.gca()

df_sales_previous.plot(kind='line',x='date',y='diff_previous',ax=ax)

plt.show()

In [ ]:
#get previous month to predict the next month